# Minimal RLOO Fine-Tuning for Code Tasks (MBPP → HumanEval)

This notebook shows a **minimal end‑to‑end** workflow to fine‑tune a small code model with **RLOO** (Reinforcement Learning with Leave‑One‑Out) on **MBPP (sanitized)** and then **evaluate on HumanEval**.

**Default model:** `HuggingFaceTB/SmolLM2-360M-Instruct` (tiny & fast).  
**Swap-in option:** Any compatible HF model id (e.g., `Qwen/Qwen2.5-Coder-1.5B-Instruct`).

> ⚠️ **Security note**: This workflow **executes model‑generated Python** when scoring with unit tests. Run in a controlled environment (e.g., Docker/VM). You can also use `evalplus` which isolates execution. 


In [2]:
# If running locally, uncomment the next line to ensure fresh packages.
%pip install -U transformers accelerate datasets trl peft safetensors human-eval evalplus bitsandbytes

   ---------------------------------------- 0.0/59.0 MB ? eta -:--:--
   -- ------------------------------------- 3.1/59.0 MB 18.5 MB/s eta 0:00:04
   ------ --------------------------------- 9.4/59.0 MB 22.6 MB/s eta 0:00:03
   ---------- ----------------------------- 15.2/59.0 MB 24.5 MB/s eta 0:00:02
   -------------- ------------------------- 21.0/59.0 MB 25.5 MB/s eta 0:00:02
   ------------------ --------------------- 26.7/59.0 MB 25.7 MB/s eta 0:00:02
   --------------------- ------------------ 32.2/59.0 MB 25.9 MB/s eta 0:00:02
   ------------------------- -------------- 37.7/59.0 MB 26.1 MB/s eta 0:00:01
   ----------------------------- ---------- 43.3/59.0 MB 26.0 MB/s eta 0:00:01
   --------------------------------- ------ 48.8/59.0 MB 26.1 MB/s eta 0:00:01
   ------------------------------------ --- 53.5/59.0 MB 26.0 MB/s eta 0:00:01
   ---------------------------------------  58.7/59.0 MB 25.8 MB/s eta 0:00:01
   ---------------------------------------- 59.0/59.0 MB 24.7 M

In [3]:
# Clean up TensorFlow packages to avoid import errors (not needed for this torch-only workflow)
%pip uninstall -y tensorflow tensorflow-intel tensorflow-io-gcs-filesystem

Note: you may need to restart the kernel to use updated packages.


In [5]:
import importlib.util
import os
print("tensorflow spec:", importlib.util.find_spec("tensorflow"))
print("HF_TOKEN present:", bool(os.environ.get("HF_TOKEN")))

tensorflow spec: None
HF_TOKEN present: True


In [7]:
from huggingface_hub import HfFolder
hf_keys = [k for k in os.environ if k.startswith("HF")]
print("Env keys starting with 'HF':", hf_keys)
print("HF_TOKEN present:", bool(os.environ.get("HF_TOKEN")))
print("HfFolder token present:", bool(HfFolder.get_token()))

Env keys starting with 'HF': ['HF_TOKEN']
HF_TOKEN present: True
HfFolder token present: True


In [8]:
import evalplus.data.mbpp as mbpp_mod
import inspect
print([name for name, obj in inspect.getmembers(mbpp_mod) if inspect.isfunction(obj)])

['_ready_mbpp_plus_path', 'completeness_check', 'get_dataset_metadata', 'get_mbpp', 'get_mbpp_plus', 'get_mbpp_plus_hash', 'make_cache', 'mbpp_deserialize_inputs', 'mbpp_serialize_inputs', 'stream_jsonl']


In [9]:
from evalplus.data.mbpp import get_mbpp
import itertools
mbpp_evalplus = get_mbpp()
print("evalplus size:", len(mbpp_evalplus))
first_key = next(iter(mbpp_evalplus))
print("Sample key:", first_key)
print(mbpp_evalplus[first_key].keys())

evalplus size: 427
Sample key: 2
dict_keys(['source_file', 'task_id', 'prompt', 'code', 'test_imports', 'test_list'])


In [10]:
import os, re, json, math, textwrap, multiprocessing as mp, queue, signal, sys
from dataclasses import dataclass
from pathlib import Path

# Disable optional back ends we do not need (avoids pulling in TensorFlow/Flax)
os.environ["TRANSFORMERS_NO_TF"] = "1"
os.environ["TRANSFORMERS_NO_FLAX"] = "1"
os.environ["USE_TF"] = "0"
os.environ["USE_FLAX"] = "0"
os.environ.setdefault("HF_HUB_DISABLE_TELEMETRY", "1")

from transformers.utils import import_utils as _hf_import_utils
_hf_import_utils.is_tf_available = lambda: False
# also update the public shortcut to be safe
from transformers import utils as _hf_utils
_hf_utils.is_tf_available = lambda: False

import torch
from datasets import load_dataset, Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
from trl import RLOOConfig, RLOOTrainer
from peft import LoraConfig

# ---- Choose your base model
MODEL_NAME = os.environ.get("BASE_MODEL", "HuggingFaceTB/SmolLM2-360M-Instruct")
# Examples:
#   "HuggingFaceTB/SmolLM2-360M-Instruct"  (≈360M)
#   "Qwen/Qwen2.5-Coder-1.5B-Instruct"     (≈1.5B)
#   "Qwen/Qwen2.5-Coder-7B-Instruct"       (heavy; consider 8-bit)

# Generation defaults for RLOO and eval
GEN_K = int(os.environ.get("RLOO_NUM_GENERATIONS", 4))     # leave-one-out over K completions
MAX_PROMPT_LEN = 512
MAX_COMPLETION_LEN = 256

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {DEVICE}")


Using device: cuda


## Load MBPP (sanitized) and build prompts
We’ll use the **sanitized** split (427 items). It includes a problem prompt, canonical solution, and unit tests. For reliable function names, we **extract only the function signature** from the canonical solution and add it to the prompt (no leakage of solution body).

In [12]:
from huggingface_hub import list_repo_files
from huggingface_hub.utils import RepositoryNotFoundError
hf_token = os.environ.get("HF_TOKEN")
try:
    # Explicitly mark this repo as a dataset; otherwise the hub looks under models and fails.
    files = list_repo_files("Muennighoff/mbpp", repo_type="dataset", token=hf_token)
    print([f for f in files if "sanitized" in f][:20])
except RepositoryNotFoundError:
    print("Cannot list 'Muennighoff/mbpp'. Accept the dataset terms and set HF_TOKEN first.")


['data/sanitized-mbpp.json']


In [19]:
from evalplus.data.mbpp import get_mbpp

# Load sanitized MBPP via evalplus (avoids gated Hugging Face script)
mbpp_dict = get_mbpp()
mbpp = Dataset.from_list(list(mbpp_dict.values()))
mbpp = mbpp.shuffle(seed=42)
print(mbpp[0].keys())
# expected fields: 'prompt', 'code', 'test_imports', 'test_list', 'challenge_test_list'

# --- helpers to build a good training prompt ---
DEF_RE = re.compile(r"^\s*def\s+\w+\s*\(.*\)\s*:")

def extract_signature(code: str) -> str:
    for line in code.splitlines():
        if DEF_RE.match(line):
            return line.strip()
    return "def solution():\n    pass"


def build_training_prompt(rec):
    sig = extract_signature(rec["code"])  # only the signature, no body
    base = rec["prompt"].strip()
    prompt = f"""
You are a Python coding assistant.
Write a **correct and efficient** solution that **defines exactly** this function signature and passes the hidden tests.

# Problem
{base}

# Function signature (must match exactly)
{sig}

# Output format
Return **only valid Python code** implementing the function. **No** markdown, comments, prints, or extra text.
""".strip()
    return prompt

# Build a compact training set for a quick run
train_size = int(os.environ.get("MBPP_TRAIN_ITEMS", 150))  # try 150 for a few minutes; increase for better results
subset = mbpp.select(range(train_size))

train_records = subset.map(lambda r: {
    "prompt_text": build_training_prompt(r),
    "test_setup": "\n".join(r.get("test_imports", []) or []),
    "tests": r.get("test_list", []) or [],
})

print("Example data:\n")
print("Prompt text:\n")
print(train_records[0]["prompt_text"])
print("\nTest setup:\n")
print(train_records[0]["test_setup"])
print("\nTests:\n")
print(train_records[0]["tests"])


dict_keys(['source_file', 'task_id', 'prompt', 'code', 'test_imports', 'test_list'])


Map: 100%|██████████| 150/150 [00:00<00:00, 4903.90 examples/s]

Example data:

Prompt text:

You are a Python coding assistant.
Write a **correct and efficient** solution that **defines exactly** this function signature and passes the hidden tests.

# Problem
Write a function to find the shared elements from the given two lists.

# Function signature (must match exactly)
def similar_elements(test_tup1, test_tup2):

# Output format
Return **only valid Python code** implementing the function. **No** markdown, comments, prints, or extra text.

Test setup:



Tests:

['assert set(similar_elements((3, 4, 5, 6),(5, 7, 4, 10))) == set((4, 5))', 'assert set(similar_elements((1, 2, 3, 4),(5, 4, 3, 7))) == set((3, 4))', 'assert set(similar_elements((11, 12, 14, 13),(17, 15, 14, 13))) == set((13, 14))']


## Reward function
The reward is **fraction of unit tests passed** (0.0–1.0) for each generated completion.  
We execute code in a **separate process with a timeout** for basic isolation. For stricter isolation, consider running this notebook inside Docker and/or using `evalplus` for evaluation time.

In [20]:
def _mp_worker(q: mp.Queue, code: str, tests: list[str], setup: str):
    """Run student code plus tests in isolation and push (passed, total) to q."""
    try:
        glb = {}
        if setup:
            exec(setup, glb, glb)
        exec(code, glb, glb)
        passed = 0
        total = len(tests)
        for t in tests:
            try:
                exec(t, glb, glb)
                passed += 1
            except Exception:
                pass
        q.put((passed, total))
    except Exception:
        q.put((0, len(tests)))


def _exec_in_subproc(code: str, tests: list[str], setup: str, timeout_s: float = 3.0):
    """Execute in a short-lived process with a timeout; return (passed, total)."""
    q = mp.Queue()
    p = mp.Process(target=_mp_worker, args=(q, code, tests, setup))
    p.start()
    p.join(timeout_s)
    if p.is_alive():
        p.terminate()
        return 0, len(tests)
    try:
        return q.get_nowait()
    except queue.Empty:
        return 0, len(tests)

# Match fenced code blocks so we can pull out raw Python when the model returns markdown.
CODE_FENCE_RE = re.compile(r"```(?:python)?\n(.*?)```", re.DOTALL)

def extract_code_only(txt: str) -> str:
    """Strip markdown fences or stray backticks; fall back to raw text."""
    m = CODE_FENCE_RE.search(txt)
    if m:
        return m.group(1).strip()
    return txt.strip().strip("`")

# TRL reward function signature: it receives completions alongside dataset columns.

def mbpp_reward(completions, tests, test_setup, **kwargs):
    """Score each completion by fraction of MBPP tests that pass."""
    outs = []
    for comp, ts, setup in zip(completions, tests, test_setup):
        # TRL may give chat-format responses (list of role/content dicts); normalize to plain text.
        if isinstance(comp, list) and len(comp) and isinstance(comp[0], dict) and "content" in comp[0]:
            comp_text = comp[0]["content"]
        else:
            comp_text = str(comp)
        code = extract_code_only(comp_text)
        passed, total = _exec_in_subproc(code, ts, setup, timeout_s=3.0)
        outs.append(0.0 if total == 0 else float(passed) / float(total))
    return outs

# quick self-check on one item
_test_code = """
"""
sc = train_records[0]
_score = mbpp_reward([
    """def noop():\n    return 0"""
], tests=[sc["tests"]], test_setup=[sc["test_setup"]])
print("example reward:", _score)


example reward: [0.0]


## Load tokenizer/model & configure RLOO (LoRA)
We keep it small and cheap: LoRA adapters on a tiny model. Increase steps / dataset size later.

In [22]:
# Tokenizer drives both prompt encoding and generation decoding during training/eval.
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# LoRA adapter hyperparameters control how much of the attention/MLP blocks we fine-tune.
peft_cfg = LoraConfig(
    r=8,  # rank of the low-rank adapters; higher values = more trainable params
    lora_alpha=16,  # scales the adapter update; affects learning capacity
    lora_dropout=0.05,  # regularizes adapter activations to curb overfitting
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"],
)

# Core RLOO loop settings determine optimization stability, sampling cost, and logging cadence.
rloo_cfg = RLOOConfig(
    output_dir="./smollm2-mbpp-rloo",  # checkpoint + log destination reused by later save/load
    learning_rate=2e-5,  # step size for adapter weights; too high destabilizes RLOO updates
    per_device_train_batch_size=2,  # number of prompts per device; limited by VRAM during sampling
    gradient_accumulation_steps=1,  # scales effective batch; increase to smooth rewards if memory allows
    num_generations=GEN_K,  # completions per prompt; defines leave-one-out pool and runtime cost
    generation_batch_size=GEN_K,  # how many generations sampled concurrently; impacts GPU RAM footprint
    max_prompt_length=MAX_PROMPT_LEN,  # truncation guard so prompts fit into context window
    max_completion_length=MAX_COMPLETION_LEN,  # cap on generated tokens; affects sampling time/test cost
    beta=0.01,  # leave-one-out baseline strength; tunes variance reduction in RLOO objective
    logging_steps=5,  # tensorboard/console logging frequency
    save_steps=50,  # checkpoint interval that ties to checkpoints reused for eval
    max_steps=int(os.environ.get("RLOO_MAX_STEPS", 60)),  # total optimizer steps; governs training duration
)

# Trainer wires model loading, reward loop, dataset, and adapter config together.
trainer = RLOOTrainer(
    model=MODEL_NAME,  # base HF model id; governs architecture capacity and tokenizer compatibility
    args=rloo_cfg,
    reward_funcs=mbpp_reward,  # custom reward defined above; shapes gradient signal
    processing_class=tokenizer,  # ensures prompts/generations reuse the tokenizer configured here
    train_dataset=train_records,  # preprocessed MBPP subset
    peft_config=peft_cfg,  # attaches LoRA adapters with the settings defined above
)
trainer


C:\Users\jkorm\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\trl\trainer\rloo_trainer.py:241: UserWarning: This trainer will soon be moved to trl.experimental and is a candidate for removal. If you rely on it and want it to remain, please share your comments here: https://github.com/huggingface/trl/issues/4223. Silence this warning by setting environment variable TRL_EXPERIMENTAL_SILENCE=1.
  warnings.warn(


### (Optional) Sanity check: generate before training

In [24]:
prompt0 = train_records[0]["prompt_text"]
inputs = tokenizer(prompt0, return_tensors="pt").to(trainer.accelerator.device)
input_len = inputs["input_ids"].shape[1]
with torch.no_grad():
    output_ids = trainer.model.generate(
        **inputs,
        max_new_tokens=192,
        do_sample=True,
        temperature=0.7,
        pad_token_id=tokenizer.pad_token_id,
    )
completion_ids = output_ids[0][input_len:]
completion = tokenizer.decode(completion_ids, skip_special_tokens=True).strip()
print("--- Prompt (truncated) ---")
print("\n".join(prompt0.splitlines()[:8]))
print("\n--- Sampled completion ---")
print(completion if completion else "[empty completion]")


--- Prompt (truncated) ---
You are a Python coding assistant.
Write a **correct and efficient** solution that **defines exactly** this function signature and passes the hidden tests.

# Problem
Write a function to find the shared elements from the given two lists.

# Function signature (must match exactly)
def similar_elements(test_tup1, test_tup2):

--- Sampled completion ---
# Example
Test cases:
[1, 2, 3, 4, 5], [2, 4, 6, 8]

**Do not** include any tests.

# Test cases
[1, 2, 3, 4, 5], [3, 5, 7, 9]

[1, 2, 3, 4, 5], [1, 4, 6, 7, 8]

[1, 2, 3, 4, 5], [1, 2, 4, 5, 6]

[1, 2, 3, 4, 5], [1, 2, 6, 8, 10]

[1, 2, 3, 4, 5


## Train (tiny demo run)
Crank up `RLOO_MAX_STEPS` and `MBPP_TRAIN_ITEMS` for real gains.

In [32]:
trainer.train()

C:\Users\jkorm\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torch\utils\checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


Step,Training Loss
5,0.000000
10,0.000000
15,0.000000
20,0.000000
25,0.000000
30,0.000000
35,0.000000
40,0.000000
45,0.000000
50,0.000000


C:\Users\jkorm\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torch\utils\checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


TrainOutput(global_step=60, training_loss=0.0, metrics={'train_runtime': 940.3565, 'train_samples_per_second': 0.128, 'train_steps_per_second': 0.064, 'total_flos': 0.0, 'train_loss': 0.0})

In [33]:
save_dir = rloo_cfg.output_dir
trainer.save_model(save_dir)
print("Saved to:", save_dir)

Saved to: ./smollm2-mbpp-rloo


## Evaluate on HumanEval (pass@1)
We’ll create a `samples.jsonl` with one completion per task, then run the official HumanEval harness.

> On Windows, prefer `pip install human-eval-windows`.
> If you want more rigorous evaluation, see `evalplus` too.


In [1]:
from datasets import load_dataset
from tqdm import tqdm

humaneval = load_dataset("openai/openai_humaneval", split="test")

# Reuse trainer.model/tokenizer in plain HF generate for speed
model = trainer.model
model.eval()

def make_completion(prompt: str, max_new_tokens=192, temperature=0.2):
    # Use plain text prompting (no chat template) to keep it simple
    inputs = tokenizer([prompt], return_tensors="pt").to(model.device)
    with torch.no_grad():
        gen_ids = model.generate(**inputs, max_new_tokens=max_new_tokens, do_sample=True,
                                 temperature=temperature, eos_token_id=tokenizer.eos_token_id)
    out = tokenizer.decode(gen_ids[0][inputs["input_ids"].shape[1]:], skip_special_tokens=True)
    return out.strip()

# Build HumanEval prompts: use the provided "prompt" field directly
samples_path = Path("./humaneval_samples.jsonl")
with samples_path.open("w", encoding="utf-8") as f:
    for row in tqdm(humaneval):
        prompt = row["prompt"]
        comp = make_completion(prompt)
        # strip code fences if present
        comp = re.sub(r"^```(?:python)?|```$", "", comp).strip()
        rec = {"task_id": row["task_id"], "completion": comp}
        f.write(json.dumps(rec) + "\n")

print("Wrote:", samples_path)


C:\Users\jkorm\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\importlib\__init__.py:126: UserWarning: A NumPy version >=1.23.5 and <2.3.0 is required for this version of SciPy (detected version 2.3.2)
  return _bootstrap._gcd_import(name[level:], package, level)
C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\importlib\__init__.py:126: UserWarning: A NumPy version >=1.23.5 and <2.3.0 is required for this version of SciPy (detected version 2.3.2)
  return _bootstrap._gcd_import(name[level:], package, level)


NameError: name 'trainer' is not defined

In [35]:
# If you hit issues on Windows, try: %pip install -U human-eval-windows && import human_eval
from human_eval.evaluation import evaluate_functional_correctness

results = evaluate_functional_correctness(str(samples_path))
print(results)
print("pass@1:", results.get("pass@1"))


Reading samples...


164it [00:00, 621.61it/s]
164it [00:00, 621.61it/s]


Running test suites...


100%|██████████| 164/164 [00:13<00:00, 11.84it/s]



Writing results to humaneval_samples.jsonl_results.jsonl...


100%|██████████| 164/164 [00:00<00:00, 15801.75it/s]

{'pass@1': np.float64(0.0)}
pass@1: 0.0


### Switching to Qwen2.5‑Coder
To try Qwen:
1. Change `MODEL_NAME` at the top to `Qwen/Qwen2.5-Coder-1.5B-Instruct`.
2. (If VRAM-limited) add `load_in_8bit=True` to the model load by exporting `BITSANDBYTES_NOWELCOME=1` and using `bnb` (already included above).
3. Increase `MAX_STEPS` and dataset size for meaningful gains.
